In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter

devices = pd.read_csv('devices.csv')
newdevices = devices.sample(n = 1048576)

my_dict = {}
to_return = []
for i in range(newdevices.shape[0]):
    to_return.append(newdevices.iloc[i]['userid'])
orders = pd.read_csv('orders.csv')
allbuyers = []
allsellers = []
buyer_list = []
seller_list = []
fraud = []
list_to_return = []

for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] == orders.iloc[i]['seller_userid']:
        fraud.append(orders.iloc[i]['buyer_userid'])
#orders.shape[0]
for i in range(orders.shape[0]):
        allbuyers.append(orders.iloc[i]['buyer_userid'])
for i in range(orders.shape[0]):
        allsellers.append(orders.iloc[i]['seller_userid'])

#check for same device
for i in range(orders.shape[0]):
    for j in range(newdevices.shape[0]):
        if allbuyers[i] == to_return[j]:
            buyer_list.append(newdevices.iloc[j]['userid'])
        elif allsellers[i] ==  to_return[j]:
            seller_list.append(newdevices.iloc[j]['userid'])

if (len(buyer_list) > len(seller_list)):
    for i in range(len(buyer_list)):
        if buyer_list[i] in seller_list:
            fraud.append(buyer_list[i])
else:
    for i in range(len(seller_list)):
        if seller_list[i] in buyer_list:
            fraud.append(seller_list[i])
            
for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] in fraud or orders.iloc[i]['seller_userid'] in fraud:
        list_to_return = [orders.iloc[i]['order_id'], 1]
    else:
        list_to_return = [orders.iloc[i]['order_id'], 0]
    
credit_cards = pd.read_csv('credit_cards.csv')

to_returncc = []
for i in range(credit_cards.shape[0]):
    to_returncc.append(credit_cards.iloc[i]['userid'])
ccfraud = []
list_to_return_others = []
for i in range(orders.shape[0]):
    for j in range(credit_cards.shape[0]):
        if allbuyers[i] == to_returncc[j]:
            buyer_list.append(credit_cards.iloc[j]['userid'])
        elif allsellers[i] ==  to_returncc[j]:
            seller_list.append(credit_cards.iloc[j]['userid'])

if (len(buyer_list) > len(seller_list)):
    for i in range(len(buyer_list)):
        if buyer_list[i] in seller_list:
            ccfraud.append(buyer_list[i])
else:
    for i in range(len(seller_list)):
        if seller_list[i] in buyer_list:
            ccfraud.append(seller_list[i])
            
for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] in ccfraud or orders.iloc[i]['seller_userid'] in ccfraud:
        list_to_return_others = [orders.iloc[i]['order_id'], 1]
    else:
        list_to_return_others = [orders.iloc[i]['order_id'], 0]
        
bank_accounts = pd.read_csv('bank_accounts.csv')
to_returnba = []
for i in range(bank_accounts.shape[0]):
    to_returnba.append(bank_accounts.iloc[i]['userid'])
print(to_returnba)

bafraud = []
list_to_return_bank_acc = []
final_list = []
for i in range(orders.shape[0]):
    for j in range(bank_accounts.shape[0]):
        if allbuyers[i] == to_returnba[j]:
            buyer_list.append(bank_accounts.iloc[j]['userid'])
        elif allsellers[i] ==  to_returnba[j]:
            seller_list.append(bank_accounts.iloc[j]['userid'])

if (len(buyer_list) > len(seller_list)):
    for i in range(len(buyer_list)):
        if buyer_list[i] in seller_list:
            bafraud.append(buyer_list[i])
else:
    for i in range(len(seller_list)):
        if seller_list[i] in buyer_list:
            bafraud.append(seller_list[i])
            
for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] in bafraud or orders.iloc[i]['seller_userid'] in bafraud:
        list_to_return_bank_acc = [orders.iloc[i]['order_id'], 1]
    else:
        list_to_return_bank_acc = [orders.iloc[i]['order_id'], 0]
        

for i in range(list_to_return):
    if list_to_return[i][1] == 0 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 0 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 0 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    else:
        final_list.append(list_to_return[i][0], 0)

workbook = xlsxwriter.Workbook('submission.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'orderid')
worksheet.write('B1', 'is_fraud')
for i in range(len(final_list)):
    worksheet.write('A'+str(i+2), final_list[0])
    worksheet.write('B' +str(i+1), final_list[1])

workbook.close()

In [81]:
orders = pd.read_csv('orders.csv')
allbuyers = []
allsellers = []
buyer_list = []
seller_list = []
fraud = []
list_to_return = []

for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] == orders.iloc[i]['seller_userid']:
        fraud.append(orders.iloc[i]['buyer_userid'])
#orders.shape[0]
for i in range(orders.shape[0]):
        allbuyers.append(orders.iloc[i]['buyer_userid'])
for i in range(orders.shape[0]):
        allsellers.append(orders.iloc[i]['seller_userid'])

#check for same device
for i in range(orders.shape[0]):
    for j in range(newdevices.shape[0]):
        if allbuyers[i] == to_return[j]:
            buyer_list.append(newdevices.iloc[j]['userid'])
        elif allsellers[i] ==  to_return[j]:
            seller_list.append(newdevices.iloc[j]['userid'])

if (len(buyer_list) > len(seller_list)):
    for i in range(len(buyer_list)):
        if buyer_list[i] in seller_list:
            fraud.append(buyer_list[i])
else:
    for i in range(len(seller_list)):
        if seller_list[i] in buyer_list:
            fraud.append(seller_list[i])
            
for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] in fraud or orders.iloc[i]['seller_userid'] in fraud:
        list_to_return = [orders.iloc[i]['order_id'], 1]
    else:
        list_to_return = [orders.iloc[i]['order_id'], 0]
    


[52616468, 9502434, 73634196]


In [83]:
credit_cards = pd.read_csv('credit_cards.csv')

to_returncc = []
for i in range(credit_cards.shape[0]):
    to_returncc.append(credit_cards.iloc[i]['userid'])
print(to_returncc)

[2579938, 2154902, 5578604, 2051728, 6216212, 7661324, 9554574, 11853094, 7965348, 7313766, 13391252, 16719792, 14188162, 10635252, 9344564, 17983302, 6171188, 13038228, 11036546, 11746202, 7718962, 2110242, 22585768, 2479350, 26184880, 5670784, 19836750, 6515170, 13633998, 10151718, 11011356, 25317864, 7231630, 32054290, 2264618, 32187876, 25311978, 27784132, 35153082, 1696434, 36747920, 18238250, 18848524, 16630356, 18226966, 37739946, 6760692, 22635706, 16570108, 19946192, 12743092, 27098360, 28948212, 42737208, 6558306, 5917576, 37267628, 19749944, 13206640, 31939816, 3694208, 33722732, 4177480, 7759016, 21847866, 12588480, 1988638, 32112398, 49433504, 39473880, 25581068, 37238300, 54965194, 52199454, 56669416, 10939196, 22479490, 6180294, 50807780, 21462474, 35192300, 43418692, 31895524, 14802574, 27369512, 42052178, 21344596, 910612, 62933662, 13703674, 68382512, 22866666, 32144034, 16230874, 32210378, 44157240, 25432150, 19044202, 15257542, 8903560, 19806744, 16700696, 68528880,

In [87]:
ccfraud = []
list_to_return_others = []
for i in range(orders.shape[0]):
    for j in range(credit_cards.shape[0]):
        if allbuyers[i] == to_returncc[j]:
            buyer_list.append(credit_cards.iloc[j]['userid'])
        elif allsellers[i] ==  to_returncc[j]:
            seller_list.append(credit_cards.iloc[j]['userid'])

if (len(buyer_list) > len(seller_list)):
    for i in range(len(buyer_list)):
        if buyer_list[i] in seller_list:
            ccfraud.append(buyer_list[i])
else:
    for i in range(len(seller_list)):
        if seller_list[i] in buyer_list:
            ccfraud.append(seller_list[i])
            
for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] in ccfraud or orders.iloc[i]['seller_userid'] in ccfraud:
        list_to_return_others = [orders.iloc[i]['order_id'], 1]
    else:
        list_to_return_others = [orders.iloc[i]['order_id'], 0]


        


IndexError: list index out of range

In [89]:
bank_accounts = pd.read_csv('bank_accounts.csv')
to_returnba = []
for i in range(bank_accounts.shape[0]):
    to_returnba.append(bank_accounts.iloc[i]['userid'])
print(to_returnba)

[21829134, 95910542, 96941876, 23452396, 12647942, 93351382, 73673972, 32609752, 97851414, 90467238, 97977206, 72355080, 37551432, 13720636, 86870650, 98834544, 99017396, 75648794, 82320120, 13035010, 80069900, 79283302, 45272194, 84946572, 14387884, 75536478, 65039678, 51258972, 65632432, 54774376, 53270042, 9987238, 101140542, 56808116, 16835614, 45776698, 53409016, 72102834, 10785438, 18711300, 43956028, 43435586, 961974, 71801502, 103377062, 59244520, 43526362, 90435948, 46211258, 90615152, 46804206, 25436306, 65033412, 46630454, 65531776, 84697866, 104807416, 102194106, 105363802, 93792902, 9163624, 26312344, 49262248, 105963684, 106111102, 105981148, 76363302, 56498328, 100482892, 16681796, 105918024, 104483358, 31102882, 84214182, 16028428, 86555032, 389946, 32544728, 108201508, 80629174, 109048822, 63163370, 37236128, 97985510, 68600374, 78558520, 13614240, 110387442, 68913622, 56269136, 105310264, 93701946, 110873650, 111182982, 60459572, 32936930, 2782722, 112302848, 43732556

C:\Users\lukas\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
bafraud = []
list_to_return_bank_acc = []
final_list = []
for i in range(orders.shape[0]):
    for j in range(bank_accounts.shape[0]):
        if allbuyers[i] == to_returnba[j]:
            buyer_list.append(bank_accounts.iloc[j]['userid'])
        elif allsellers[i] ==  to_returnba[j]:
            seller_list.append(bank_accounts.iloc[j]['userid'])

if (len(buyer_list) > len(seller_list)):
    for i in range(len(buyer_list)):
        if buyer_list[i] in seller_list:
            bafraud.append(buyer_list[i])
else:
    for i in range(len(seller_list)):
        if seller_list[i] in buyer_list:
            bafraud.append(seller_list[i])
            
for i in range(orders.shape[0]):
    if orders.iloc[i]['buyer_userid'] in bafraud or orders.iloc[i]['seller_userid'] in bafraud:
        list_to_return_bank_acc = [orders.iloc[i]['order_id'], 1]
    else:
        list_to_return_bank_acc = [orders.iloc[i]['order_id'], 0]
        

for i in range(list_to_return):
    if list_to_return[i][1] == 0 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 0 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 0 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 1:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 0 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    elif list_to_return[i][1] == 1 and list_to_return_others[i][1] == 1 and list_to_return_bank_acc[i][1] == 0:
        final_list.append(list_to_return[i][0], 1)
    else:
        final_list.append(list_to_return[i][0], 0)

workbook = xlsxwriter.Workbook('submission.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'orderid')
worksheet.write('B1', 'is_fraud')
for i in range(len(final_list)):
    worksheet.write('A'+str(i+2), final_list[0])
    worksheet.write('B' +str(i+1), final_list[1])

workbook.close()